# Real Box Office Data Analysis

This project analyzes a set of real box office data (provided by flatiron data science school) which includes information gathered from Box Office Mojo, IMDB, Rotten Tomatoes, ThemovieDB, and The Numbers in order to provide business insights for the investors looking to open a new movie studio.